<a href="https://colab.research.google.com/github/AlessandroFornasier/diffusion-models/blob/main/autoencoders/vae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn

from collections import OrderedDict
from dataclasses import dataclass
from typing import List, Optional


@dataclass
class AutoencoderState:
  """
  Autoencoder state.

  Attributes:
   - x (Optional[torch.Tensor]): Input data
   - z (Optional[torch.Tensor]): Latent space sample
   - x_hat (Optional[torch.Tensor]): Reconstructed data
  """
  x : Optional[torch.Tensor] = None                         # Input
  z : Optional[torch.Tensor] = None                         # Latent space sample
  x_hat : Optional[torch.Tensor] = None                     # Reconstructed input


class Autoencoder(nn.Module):
  """
  Autoencoder class.

  Args:
    encoder (OrderedDict[str, nn.Module]): Ordered dictionary of encoder layers
    decoder (OrderedDict[str, nn.Module]): Ordered dictionary of decoder layers
  """

  def __init__(
    self,
    encoder: OrderedDict[str, nn.Module],
    decoder: OrderedDict[str, nn.Module]
  ) -> None:
    super(Autoencoder, self).__init__()

    self.encoder = nn.Sequential(encoder)
    self.decoder = nn.Sequential(decoder)

    def encode(self, x) -> torch.Tensor:
      """
      Encodes the input data into the latent space.

      Args:
        x (torch.Tensor): Input data.

      Returns:
        z (torch.Tensor): Encoded data, latent space.
      """
      return self.encoder(x)

    def decode(self, z) -> torch.Tensor:
      """
      Decodes the latent space data.

      Args:
        z (torch.Tensor): Latent space data.

      Returns:
        x_hat (torch.Tensor): Decoded data, output space.
      """
      return self.decoder(z)

    def forward(self, x) -> torch.Tensor:
      """
      Forward pass of the autoencoder
      """
      state = AutoencoderState(x)
      state.z = self.encode(x)
      state.x_hat = self.decode(state.z)
      return state


@dataclass
class VAEState(AutoencoderState):
  """
  Variational autoencoder state.

  Attributes:
   - x (Optional[torch.Tensor]): Input data
   - z (Optional[torch.Tensor]): Latent space sample
   - x_hat (Optional[torch.Tensor]): Reconstructed data
   - dist (Optional[torch.distributions.Distribution]): Encoder Gaussian distribution
  """
  dist : Optional[torch.distributions.Distribution] = None  # Latent space distribution


class VAE(Autoencoder):
  """
  Variational autoencoder class.

  Args:
    encoder (OrderedDict[str, nn.Module]): Ordered dictionary of encoder layers
    decoder (OrderedDict[str, nn.Module]): Ordered dictionary of decoder layers
  """

  def __init__(
    self,
    encoder: OrderedDict[str, nn.Module],
    decoder: OrderedDict[str, nn.Module]
  ) -> None:
    super().__init__(encoder, decoder)

    self.softplus = nn.Softplus()

  def encode(self, x, eps: float = 1e-6) -> torch.distributions.Distribution:
    """
    Encodes the input data into the latent space.

    Args:
      x (torch.Tensor): Input data.
      eps (float): Small value to avoid numerical instability.

    Returns:
      dist (torch.distributions.MultivariateNormal): Normal distribution of the encoded data.

    Note:
      Learning logvar improves numerical stability since var is smaller than zero and tipically smaller than once. Hence logvar is within (-inf, log(1)).
      Softplus + epsilon (softplus(x) = \log(1 + \exp(x))) is used to get sigma instead of directly exponentiating while ensuring numerical stability
    """
    e = self.encoder(x)
    mu, logvar = torch.tensor_split(e, 2, dim=-1)
    var = self.softplus(logvar) + eps
    return torch.distributions.MultivariateNormal(mu, scale_tril=torch.diag_embed(var)) # Use scale_tril as it is more efficient

  def reparametrize(self, dist) -> torch.Tensor:
    """
    Perform sampling via the reparametrization trick

    Args:
      dist (torch.distributions.MultivariateNormal): Normal distribution of the encoded data.

    Returns:
      z (torch.Tensor): Sampled data from the latent space z = mu + sigma * epsilon. With epsilon ~ N(0,I)
    """
    return dist.rsample()

  def decode(self, z) -> torch.Tensor:
    """
    Decodes the data from the latent space to the original input space.

    Args:
      z (torch.Tensor): Data in the latent space.

    Returns:
      x_hat (torch.Tensor): Reconstructed data in the original input space.
    """
    return self.decoder(z)

  def forward(self, x) -> VAEState:
    """
    Performs a forward pass of the VAE.

    Args:
      x (torch.Tensor): Input data.

    Returns:
      state (VAEState): state of the VAE.
    """
    state = VAEState(x)
    state.dist = self.encode(state.x)
    state.z = self.reparametrize(state.dist)
    state.x_hat = self.decode(state.z)
    return state

In [11]:
import os

from tqdm.auto import tqdm
from torch.optim import Optimizer
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from typing import Callable, Dict


class Trainer:
  """
  Trainer class for training an autoencoder model.

  Args:
    device (torch.device): The device (CPU or GPU) to run the model on.
    model (nn.Module): The autoencoder model to be trained.
    loss (Callable[[AutoencoderState], Dict[torch.Tensor]]): A callable loss function that takes the model's state and returns a scalar loss value.
    optimizer (Optimizer): The optimizer for training the model.
    epochs (int): Number of epochs
    writer (Optional[SummaryWriter]): Optional TensorBoard writer for logging training metrics.
  """
  def __init__(
    self,
    device: torch.device,
    model: nn.Module,
    loss: Callable[[AutoencoderState], Dict[str, torch.Tensor]],
    optimizer: Optimizer,
    epochs: int,
    writer: Optional[SummaryWriter] = None,
  ) -> None:
    self.device = device
    self.model = model.to(self.device)
    self.loss = loss
    self.optimizer = optimizer
    self.epochs = epochs
    self.writer = writer
    self.models_folder = './models'

  def train(self, dataloader: DataLoader) -> None:
    """
    Trains the autoencoder model on the given dataset.

    Args:
      dataloader (DataLoader): The DataLoader for loading training data.
    """
    self.model.train()

    step = 0
    for epoch in range(self.epochs):
      epoch_loss = 0.0
      progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")

      for batch_idx, (data, _) in enumerate(progress_bar):
        data = data.to(self.device)

        state = self.model(data)
        losses = self.loss(state)

        loss = sum(losses.values())

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        batch_loss = loss.item()
        epoch_loss += batch_loss
        average_loss = epoch_loss / (batch_idx + 1)
        progress_bar.set_postfix(loss=batch_loss)

        if self.writer:
          self.writer.add_scalar("Train/Loss/Batch", batch_loss, step)
          self.writer.add_scalar("Train/Loss/Epoch", average_loss, step)
          for name, loss in losses.items():
            self.writer.add_scalar(f"Train/Loss/{name}", loss, step)

        step += 1
        print(f"Epoch [{epoch+1}/{epochs}] - Batch loss: {batch_loss:.4f} - Epoch Loss: {epoch_loss:.4f} - Avg Loss: {average_loss:.4f}")

  def train_and_save(self, dataloader: DataLoader, path: str, name: str):
    """
    Trains the autoencoder model on the given dataset and save the model weight.

    Args:
      dataloader (DataLoader): The DataLoader for loading training data.
      path (str): Path to save the model weights
      name (str): name of the model weights
    """
    self.train(dataloader=dataloader)
    os.makedirs(path, exist_ok=True)
    torch.save(self.model.state_dict(), f'{path}/{name}')
    print(f'Model saved to: {path}/{name}')

  def test(self, dataloader: DataLoader) -> None:
    """
    Test the autoencoder model on the given dataset.

    Args:
      dataloader (DataLoader): The DataLoader for loading training data.
    """
    self.model.eval()

    average_loss = 0.0
    with torch.no_grad():
      for data, _ in tqdm(dataloader, desc="Testing"):
        data = data.to(self.device)

        state = self.model(data)
        losses = self.loss(state)
        loss = sum(losses.values())

        average_loss += loss.item()

    average_loss /= len(dataloader)
    if self.writer:
      self.writer.add_scalar("Test/Loss/Average", average_loss)
      for name, loss in losses.items():
        self.writer.add_scalar(f"Test/Loss/Average/{name}", loss / len(dataloader))

    print(f"Average test loss: {average_loss:.4f}")

  def load_and_test(self, dataloader: DataLoader, path: str, name: str):
    """
    Test the autoencoder model on the given dataset.

    Args:
      dataloader (DataLoader): The DataLoader for loading training data.
      path (str): Path relative to the model folder where the model weights are saved
      name (str): name of the model weights
    """
    self.model.load_state_dict(torch.load(f'{path}/{name}'))
    self.test(dataloader=dataloader)

In [5]:
from torchvision import datasets
from torchvision.transforms import v2


class MNISTLoader:
  """
  A utility class for loading and preprocessing the MNIST dataset using a custom transformation pipeline.

  The transformation pipeline includes:
    - Conversion to a PyTorch image tensor.
    - Scaling pixel values from [0, 255] to [0.0, 1.0].
    - Flattening the image into a 1D tensor.

  Args:
    batch_size (int): Number of samples per batch in the DataLoader.
  """
  def __init__(self, batch_size: int) -> None:
    self.batch_size = batch_size
    self.transform = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True), v2.Lambda(lambda x: x.view(-1))])

  def get_dataloader(self, train: bool) -> DataLoader:
    data = datasets.MNIST('./data/MNIST', download=True, train=train, transform=self.transform)
    return DataLoader(data, batch_size=self.batch_size, shuffle=True)

In [6]:
import torch.nn.functional as Func


class VAELoss:
  """
  VAE Loss callable class. The VAE loss is given by the ELBO,
  which is the the sum of the reconstruction loss and the KL divergence loss

  Note:
    A VAE is trained by maximizing ELBO:
    - Reconstruction loss (MSE ~ cross entropy)
    - KL divergence

  Refernce:
    - https://hunterheidenreich.com/posts/modern-variational-autoencoder-in-pytorch/
    - https://github.com/pytorch/examples/blob/main/vae/main.py
  """
  def __init__(self, binary: bool) -> None:
    self.binary = binary

  def __binary_vae_loss(self, state: VAEState) -> Dict[str, torch.Tensor]:
    rl = Func.binary_cross_entropy(state.x_hat, state.x, reduction='none').sum(-1).mean() # Reconstruction loss
    target_dist = torch.distributions.MultivariateNormal(
      torch.zeros_like(state.z, device=state.z.device),
      scale_tril=torch.eye(state.z.shape[-1], device=state.z.device).unsqueeze(0).expand(state.z.shape[0], -1, -1),
    )
    kll = torch.distributions.kl.kl_divergence(state.dist, target_dist).mean() # KL loss
    return {"Reconstruction": rl, "KL": kll}

  def __call__(self, state: VAEState) -> Dict[str, torch.Tensor]:
    if self.binary:
      return self.__binary_vae_loss(state)
    else:
      raise NotImplementedError

In [7]:
from datetime import datetime

def vae_encoder_decoder(dims: List[int], binary: bool):
  """
  Defines the VAE encoder and decoder structure.

  Args:
    dims (List[int]): List of dimesions of the autoencoder layers
    binary (bool): Flag to indicate whether the autoencoder processes binary data [0, 1]

  Note:
    The output of the final encoder layer has double the size for mean and variance and no activation.
    If binary the activation of the last decoder layer is a Sigmoid to normalize to (0, 1)
  """
  encoder = []
  for n, (idim, odim) in enumerate(zip(dims[:-2], dims[1:-1])):
    encoder.append((f'Linear_{n}', nn.Linear(idim, odim)))
    encoder.append((f'SiLU_{n}', nn.SiLU()))
  encoder.append((f'Linear_{len(dims) - 2}', nn.Linear(dims[-2], 2 * dims[-1])))

  decoder = []
  for n, (idim, odim) in enumerate(zip(dims[-1:1:-1], dims[-2:0:-1])):
    decoder.append((f'Linear_{n}', nn.Linear(idim, odim)))
    decoder.append((f'SiLU_{n}', nn.SiLU()))
  decoder.append((f'Linear_{len(dims) - 2}', nn.Linear(dims[1], dims[0])))

  if binary:
    decoder.append((f'Sigmoid_{len(dims) - 2}', nn.Sigmoid()))

  return encoder, decoder

timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
model_path = f'./models/MNIST'
model_name = f'VAE_{timestamp}.pt'

binary = True
dims = [28*28, 512, 128, 64, 24, 12, 6, 2]
learning_rate = 1e-3
weight_decay = 1e-2
epochs = 50

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder, decoder = vae_encoder_decoder(dims=dims, binary=binary)
model = VAE(encoder=OrderedDict(encoder), decoder=OrderedDict(decoder))
loss = VAELoss(binary=binary)
dataloader = MNISTLoader(batch_size=128)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
writer = SummaryWriter(f'./runs/MNIST/VAE_{timestamp}')
trainer = Trainer(device=device, model=model, loss=loss, optimizer=optimizer, epochs=epochs, writer=writer)

trainer.train_and_save(dataloader.get_dataloader(train=True), model_path, model_name)
writer.flush()
writer.close()

100%|██████████| 9.91M/9.91M [00:00<00:00, 131MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 29.7MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 95.2MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 2.98MB/s]


Epoch 1/50:   0%|          | 0/469 [00:00<?, ?it/s]

Epoch [1/50] - Batch loss: 544.0444 - Epoch Loss: 544.0444 - Avg Loss: 544.0444
Epoch [1/50] - Batch loss: 539.9706 - Epoch Loss: 1084.0150 - Avg Loss: 542.0075
Epoch [1/50] - Batch loss: 535.6476 - Epoch Loss: 1619.6625 - Avg Loss: 539.8875
Epoch [1/50] - Batch loss: 530.8463 - Epoch Loss: 2150.5089 - Avg Loss: 537.6272
Epoch [1/50] - Batch loss: 525.0850 - Epoch Loss: 2675.5938 - Avg Loss: 535.1188
Epoch [1/50] - Batch loss: 518.9064 - Epoch Loss: 3194.5002 - Avg Loss: 532.4167
Epoch [1/50] - Batch loss: 511.5636 - Epoch Loss: 3706.0638 - Avg Loss: 529.4377
Epoch [1/50] - Batch loss: 503.3069 - Epoch Loss: 4209.3707 - Avg Loss: 526.1713
Epoch [1/50] - Batch loss: 493.8606 - Epoch Loss: 4703.2313 - Avg Loss: 522.5813
Epoch [1/50] - Batch loss: 481.1222 - Epoch Loss: 5184.3534 - Avg Loss: 518.4353
Epoch [1/50] - Batch loss: 469.3226 - Epoch Loss: 5653.6760 - Avg Loss: 513.9705
Epoch [1/50] - Batch loss: 455.4171 - Epoch Loss: 6109.0931 - Avg Loss: 509.0911
Epoch [1/50] - Batch loss: 44

Epoch 2/50:   0%|          | 0/469 [00:00<?, ?it/s]

Epoch [2/50] - Batch loss: 187.9804 - Epoch Loss: 187.9804 - Avg Loss: 187.9804
Epoch [2/50] - Batch loss: 191.1511 - Epoch Loss: 379.1315 - Avg Loss: 189.5658
Epoch [2/50] - Batch loss: 196.8809 - Epoch Loss: 576.0124 - Avg Loss: 192.0041
Epoch [2/50] - Batch loss: 197.4229 - Epoch Loss: 773.4353 - Avg Loss: 193.3588
Epoch [2/50] - Batch loss: 193.0402 - Epoch Loss: 966.4755 - Avg Loss: 193.2951
Epoch [2/50] - Batch loss: 188.5405 - Epoch Loss: 1155.0160 - Avg Loss: 192.5027
Epoch [2/50] - Batch loss: 193.7181 - Epoch Loss: 1348.7341 - Avg Loss: 192.6763
Epoch [2/50] - Batch loss: 196.2276 - Epoch Loss: 1544.9617 - Avg Loss: 193.1202
Epoch [2/50] - Batch loss: 194.7505 - Epoch Loss: 1739.7122 - Avg Loss: 193.3014
Epoch [2/50] - Batch loss: 192.3071 - Epoch Loss: 1932.0194 - Avg Loss: 193.2019
Epoch [2/50] - Batch loss: 190.6797 - Epoch Loss: 2122.6991 - Avg Loss: 192.9726
Epoch [2/50] - Batch loss: 178.4820 - Epoch Loss: 2301.1810 - Avg Loss: 191.7651
Epoch [2/50] - Batch loss: 190.17

...

Epoch [48/50] - Batch loss: 158.5071 - Epoch Loss: 158.5071 - Avg Loss: 158.5071
Epoch [48/50] - Batch loss: 161.6040 - Epoch Loss: 320.1111 - Avg Loss: 160.0555
Epoch [48/50] - Batch loss: 158.9691 - Epoch Loss: 479.0802 - Avg Loss: 159.6934
Epoch [48/50] - Batch loss: 161.9844 - Epoch Loss: 641.0647 - Avg Loss: 160.2662
Epoch [48/50] - Batch loss: 162.3461 - Epoch Loss: 803.4108 - Avg Loss: 160.6822
Epoch [48/50] - Batch loss: 170.6546 - Epoch Loss: 974.0654 - Avg Loss: 162.3442
Epoch [48/50] - Batch loss: 167.4651 - Epoch Loss: 1141.5306 - Avg Loss: 163.0758
Epoch [48/50] - Batch loss: 164.0457 - Epoch Loss: 1305.5762 - Avg Loss: 163.1970
Epoch [48/50] - Batch loss: 160.7853 - Epoch Loss: 1466.3615 - Avg Loss: 162.9291
Epoch [48/50] - Batch loss: 160.8531 - Epoch Loss: 1627.2146 - Avg Loss: 162.7215
Epoch [48/50] - Batch loss: 159.3993 - Epoch Loss: 1786.6140 - Avg Loss: 162.4195
Epoch [48/50] - Batch loss: 166.2373 - Epoch Loss: 1952.8513 - Avg Loss: 162.7376
Epoch [48/50] - Batch 

Epoch 49/50:   0%|          | 0/469 [00:00<?, ?it/s]

Epoch [49/50] - Batch loss: 166.9552 - Epoch Loss: 166.9552 - Avg Loss: 166.9552
Epoch [49/50] - Batch loss: 156.5035 - Epoch Loss: 323.4586 - Avg Loss: 161.7293
Epoch [49/50] - Batch loss: 158.9341 - Epoch Loss: 482.3927 - Avg Loss: 160.7976
Epoch [49/50] - Batch loss: 167.3316 - Epoch Loss: 649.7243 - Avg Loss: 162.4311
Epoch [49/50] - Batch loss: 162.1740 - Epoch Loss: 811.8983 - Avg Loss: 162.3797
Epoch [49/50] - Batch loss: 155.4570 - Epoch Loss: 967.3552 - Avg Loss: 161.2259
Epoch [49/50] - Batch loss: 176.4648 - Epoch Loss: 1143.8200 - Avg Loss: 163.4029
Epoch [49/50] - Batch loss: 158.5021 - Epoch Loss: 1302.3221 - Avg Loss: 162.7903
Epoch [49/50] - Batch loss: 162.8607 - Epoch Loss: 1465.1827 - Avg Loss: 162.7981
Epoch [49/50] - Batch loss: 169.9493 - Epoch Loss: 1635.1320 - Avg Loss: 163.5132
Epoch [49/50] - Batch loss: 165.1246 - Epoch Loss: 1800.2566 - Avg Loss: 163.6597
Epoch [49/50] - Batch loss: 163.5304 - Epoch Loss: 1963.7869 - Avg Loss: 163.6489
Epoch [49/50] - Batch 

Epoch 50/50:   0%|          | 0/469 [00:00<?, ?it/s]

Epoch [50/50] - Batch loss: 163.5560 - Epoch Loss: 163.5560 - Avg Loss: 163.5560
Epoch [50/50] - Batch loss: 163.6533 - Epoch Loss: 327.2093 - Avg Loss: 163.6047
Epoch [50/50] - Batch loss: 155.6115 - Epoch Loss: 482.8208 - Avg Loss: 160.9403
Epoch [50/50] - Batch loss: 162.3163 - Epoch Loss: 645.1371 - Avg Loss: 161.2843
Epoch [50/50] - Batch loss: 156.9595 - Epoch Loss: 802.0966 - Avg Loss: 160.4193
Epoch [50/50] - Batch loss: 170.4737 - Epoch Loss: 972.5703 - Avg Loss: 162.0950
Epoch [50/50] - Batch loss: 157.3401 - Epoch Loss: 1129.9103 - Avg Loss: 161.4158
Epoch [50/50] - Batch loss: 162.9554 - Epoch Loss: 1292.8657 - Avg Loss: 161.6082
Epoch [50/50] - Batch loss: 157.5095 - Epoch Loss: 1450.3752 - Avg Loss: 161.1528
Epoch [50/50] - Batch loss: 165.7350 - Epoch Loss: 1616.1102 - Avg Loss: 161.6110
Epoch [50/50] - Batch loss: 158.2775 - Epoch Loss: 1774.3876 - Avg Loss: 161.3080
Epoch [50/50] - Batch loss: 159.0066 - Epoch Loss: 1933.3942 - Avg Loss: 161.1162
Epoch [50/50] - Batch 

In [12]:
from google.colab import files

!zip -r data.zip ./data
files.download('data.zip')

!zip -r models.zip ./models
files.download('models.zip')

!zip -r logs.zip ./runs
files.download('logs.zip')


  adding: models/ (stored 0%)
  adding: models/MNIST/ (stored 0%)
  adding: models/MNIST/VAE_20250421073911.pt (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>